In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

# Input Data
sectors = ['Energy', 'Industrials', 'Information Technology', 'Utilities', 'Health Care', 'Financials']
benchmark_weights = np.array([0.034575, 0.087423, 0.311420, 0.024957, 0.107466, 0.157911])
underweight_matrix = np.array([
    [0.089618, 0.025347, 0.011072, 0.015178, 0.008784, 0.028007],
    [0.025347, 0.030405, 0.029316, 0.017300, 0.015910, 0.029440],
    [0.011072, 0.029316, 0.051999, 0.015003, 0.019991, 0.029051],
    [0.015178, 0.017300, 0.015003, 0.033967, 0.016893, 0.018864],
    [0.008784, 0.015910, 0.019991, 0.016893, 0.022112, 0.018360],
    [0.028007, 0.029440, 0.029051, 0.018864, 0.018360, 0.036468]
])
active_share_target = 0.5
initial_guess = np.full(len(benchmark_weights), 0.5 / len(benchmark_weights))

# Active Risk Calculation
def calculate_active_risk(weights, covariance_matrix):
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return np.sqrt(portfolio_variance)

# Marginal Risk Contribution (Gradients)
def marginal_risk_contribution(weights, covariance_matrix):
    return np.dot(covariance_matrix, weights)

# Objective Function: Minimize Variance Contribution Std
def objective(weights):
    active_risk = calculate_active_risk(weights, underweight_matrix)
    marginal_risk = marginal_risk_contribution(weights, underweight_matrix) / active_risk
    variance_contribution = (marginal_risk * weights) / active_risk
    return np.sum((variance_contribution[variance_contribution != 0] - (1/np.count_nonzero(weights))) ** 2)

# Track Iterations, Gradients, Active Weights, Variance Contributions
iterations, absolute_weights, gradients, active_weights, variance_contributions, std_values, active_share_values = [], [], [], [], [], [], []
iterations.append(initial_guess)
absolute_weights.append(benchmark_weights - initial_guess)
active_risk = calculate_active_risk(initial_guess, underweight_matrix)
marginal_risk = marginal_risk_contribution(initial_guess, underweight_matrix) / active_risk
active_weights.append(initial_guess)
variance_contributions.append((marginal_risk * initial_guess) / active_risk)
vc = (marginal_risk * initial_guess) / active_risk
std_values.append(np.sum((vc[vc != 0] - (1/np.count_nonzero(initial_guess))) ** 2))
active_share_values.append(np.sum(initial_guess))
def callback(weights):
    iterations.append(weights.copy())
    absolute_weights.append(benchmark_weights - weights)
    gradients.append(marginal_risk_contribution(weights, underweight_matrix))
    active_weights.append(weights)
    active_risk = calculate_active_risk(weights, underweight_matrix)
    marginal_risk = marginal_risk_contribution(weights, underweight_matrix) / active_risk
    variance_contributions.append((marginal_risk * weights) / active_risk)
    vc = (marginal_risk * weights) / active_risk
    std_values.append(np.sum((vc[vc != 0] - (1/np.count_nonzero(weights))) ** 2))
    active_share_values.append(np.sum(weights))  


# Constraints
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x)  - active_share_target}]
bounds = [(0, _) for _ in benchmark_weights]

# Optimization
minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints, callback=callback,tol=1e-6)

max_y_values = [max(np.max(active_weights[i] * 100), np.max(variance_contributions[i] * 100)) for i in range(len(iterations))]

# Prepare Data for Plotly
iterations, active_weights, variance_contributions = map(np.array, [iterations, active_weights, variance_contributions])
frames = [
    go.Frame(
        data=[
            go.Bar(name="Active Weight (%)", x=sectors, y=active_weights[i] * 100, text=active_weights[i] * 100, 
                   texttemplate='%{text:.2f}%', textposition='outside', marker_color="orange"),
            go.Bar(name="Absolute Weight (%)", x=sectors, y=absolute_weights[i] * 100, text=absolute_weights[i] * 100, 
                   texttemplate='%{text:.2f}%', textposition='outside', marker_color="green"),
            go.Bar(name="Ex Ante Risk Contribution (%)", x=sectors, y=variance_contributions[i] * 100, 
                   text=variance_contributions[i] * 100, texttemplate='%{text:.2f}%', textposition='outside',
                   marker_color="blue")
        ],
        name=f"Iteration {i+1}",
        layout=go.Layout(
            annotations=[
                dict(
                    text=f"Sum of Squared Differences: {std_values[i]:.4f} | Active Share: {active_share_values[i]:.4f}",
                    x=0.5,
                    y=-0.2,
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14)
                )
            ]
        )
    ) for i in range(len(iterations))
]

# Plotly Figure
fig = go.Figure(
    data=[
        go.Bar(name="Active Weight (%)", x=sectors, y=active_weights[0] * 100, text=active_weights[0] * 100, 
               texttemplate='%{text:.2f}%', textposition='outside', marker_color="orange"),
        go.Bar(name="Absolute Weight (%)", x=sectors, y=absolute_weights[0] * 100, text=absolute_weights[0] * 100, 
               texttemplate='%{text:.2f}%', textposition='outside', marker_color="green"),
        go.Bar(name="Ex Ante Risk Contribution (%)", x=sectors, y=variance_contributions[0] * 100, 
               text=variance_contributions[0] * 100, texttemplate='%{text:.2f}%', textposition='outside',
               marker_color="blue")
    ],
    layout=go.Layout(
        title="Optimization Process: Active Weights and Ex Ante Risk Contribution",
        barmode='group',
        yaxis=dict(range=[-50, 50]),
        sliders=[{
            "steps": [
                {"method": "animate", "args": [[f"Iteration {i+1}"], {"frame": {"duration": 0, "redraw": True}}],
                 "label": f"Iteration {i+1}"} for i in range(len(frames))
            ]
        }]
    ),
    frames=frames
)

# Export HTML with Barclays Styling and MathJax
plotly_html = fig.to_html(full_html=False, include_plotlyjs='cdn')

In [206]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

# Input Data
sectors = ['Energy', 'Industrials', 'Information Technology', 'Utilities', 'Health Care', 'Financials']
benchmark_weights = np.array([0.034575, 0.087423, 0.311420, 0.024957, 0.107466, 0.157911])
underweight_matrix = np.array([
    [0.089618, 0.025347, 0.011072, 0.015178, 0.008784, 0.028007],
    [0.025347, 0.030405, 0.029316, 0.017300, 0.015910, 0.029440],
    [0.011072, 0.029316, 0.051999, 0.015003, 0.019991, 0.029051],
    [0.015178, 0.017300, 0.015003, 0.033967, 0.016893, 0.018864],
    [0.008784, 0.015910, 0.019991, 0.016893, 0.022112, 0.018360],
    [0.028007, 0.029440, 0.029051, 0.018864, 0.018360, 0.036468]
])
active_share_target = 0.5
initial_guess = np.full(len(benchmark_weights), 0.5 / len(benchmark_weights))

# Active Risk Calculation
def calculate_active_risk(weights, covariance_matrix):
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return np.sqrt(portfolio_variance)

# Marginal Risk Contribution (Gradients)
def marginal_risk_contribution(weights, covariance_matrix):
    return np.dot(covariance_matrix, weights)

# Objective Function: Minimize Variance Contribution Std
def objective(weights):
    active_risk = calculate_active_risk(weights, underweight_matrix)
    marginal_risk = marginal_risk_contribution(weights, underweight_matrix) / active_risk
    variance_contribution = (marginal_risk * weights) / active_risk
    return np.sum((variance_contribution[variance_contribution != 0] - (1/np.count_nonzero(weights))) ** 2)

# Track Iterations, Gradients, Active Weights, Variance Contributions
iterations, absolute_weights, gradients, active_weights, variance_contributions, std_values, active_share_values = [], [], [], [], [], [], []
iterations.append(initial_guess)
absolute_weights.append(benchmark_weights - initial_guess)
active_risk = calculate_active_risk(initial_guess, underweight_matrix)
marginal_risk = marginal_risk_contribution(initial_guess, underweight_matrix) / active_risk
active_weights.append(initial_guess)
variance_contributions.append((marginal_risk * initial_guess) / active_risk)
vc = (marginal_risk * initial_guess) / active_risk
std_values.append(np.sum((vc[vc != 0] - (1/np.count_nonzero(initial_guess))) ** 2))
active_share_values.append(np.sum(initial_guess))
def callback(weights):
    iterations.append(weights.copy())
    absolute_weights.append(benchmark_weights - weights)
    gradients.append(marginal_risk_contribution(weights, underweight_matrix))
    active_weights.append(weights)
    active_risk = calculate_active_risk(weights, underweight_matrix)
    marginal_risk = marginal_risk_contribution(weights, underweight_matrix) / active_risk
    variance_contributions.append((marginal_risk * weights) / active_risk)
    vc = (marginal_risk * weights) / active_risk
    std_values.append(np.sum((vc[vc != 0] - (1/np.count_nonzero(weights))) ** 2))
    active_share_values.append(np.sum(weights))  


# Constraints
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x)  - active_share_target}]
bounds = [(0, 1) for _ in benchmark_weights]

# Optimization
minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints, callback=callback,tol=1e-6)

max_y_values = [max(np.max(active_weights[i] * 100), np.max(variance_contributions[i] * 100)) for i in range(len(iterations))]

# Prepare Data for Plotly
iterations, active_weights, variance_contributions = map(np.array, [iterations, active_weights, variance_contributions])
frames = [
    go.Frame(
        data=[
            go.Bar(name="Active Weight (%)", x=sectors, y=active_weights[i] * 100, text=active_weights[i] * 100, 
                   texttemplate='%{text:.2f}%', textposition='outside', marker_color="orange"),
            go.Bar(name="Absolute Weight (%)", x=sectors, y=absolute_weights[i] * 100, text=absolute_weights[i] * 100, 
                   texttemplate='%{text:.2f}%', textposition='outside', marker_color="green"),
            go.Bar(name="Ex Ante Risk Contribution (%)", x=sectors, y=variance_contributions[i] * 100, 
                   text=variance_contributions[i] * 100, texttemplate='%{text:.2f}%', textposition='outside',
                   marker_color="blue")
        ],
        name=f"Iteration {i+1}",
        layout=go.Layout(
            annotations=[
                dict(
                    text=f"Sum of Squared Differences: {std_values[i]:.4f} | Active Share: {active_share_values[i]:.4f}",
                    x=0.5,
                    y=-0.2,
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14)
                )
            ]
        )
    ) for i in range(len(iterations))
]

# Plotly Figure
fig = go.Figure(
    data=[
        go.Bar(name="Active Weight (%)", x=sectors, y=active_weights[0] * 100, text=active_weights[0] * 100, 
               texttemplate='%{text:.2f}%', textposition='outside', marker_color="orange"),
        go.Bar(name="Absolute Weight (%)", x=sectors, y=absolute_weights[0] * 100, text=absolute_weights[0] * 100, 
               texttemplate='%{text:.2f}%', textposition='outside', marker_color="green"),
        go.Bar(name="Ex Ante Risk Contribution (%)", x=sectors, y=variance_contributions[0] * 100, 
               text=variance_contributions[0] * 100, texttemplate='%{text:.2f}%', textposition='outside',
               marker_color="blue")
    ],
    layout=go.Layout(
        title="Optimization Process: Active Weights and Ex Ante Risk Contribution Allowing for Shorts",
        barmode='group',
        yaxis=dict(range=[-50, 50]),
        sliders=[{
            "steps": [
                {"method": "animate", "args": [[f"Iteration {i+1}"], {"frame": {"duration": 0, "redraw": True}}],
                 "label": f"Iteration {i+1}"} for i in range(len(frames))
            ]
        }]
    ),
    frames=frames
)

# Export HTML with Barclays Styling and MathJax
plotly_html_short = fig.to_html(full_html=False, include_plotlyjs='cdn')

In [207]:
initial_guess

array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333])

In [38]:


html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Optimization Visualization</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-AMS_CHTML"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            color: #333;
            margin: 0;
            padding: 20px;
        }}
        h1, h2, h3 {{
            color: #0078D7;
        }}
        .content {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 4px 6px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }}
        p, ul {{
            line-height: 1.6;
        }}
        ul {{
            padding-left: 20px;
        }}
        .equation {{
            text-align: center;
            margin: 10px 0;
        }}
    </style>
</head>
<body>
    <div class="content">
        <h1>Mathematical Explanation of the Risk Distribution Optimization</h1>

        <h2>Ex Ante Active Risk</h2>
        <p>
            The <strong>Ex Ante Active Risk</strong> (used interchangeably with Ex Ante Tracking Error) measures the portfolio's <strong>expected</strong>  deviation from the benchmark. It is defined as:
        </p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Tracking Error}} = \\sqrt{{(w - w_b)^T \\Sigma (w - w_b)}}
            \\]
        </div>
        <p>Where:</p>
        <ul>
            <li><strong>\( w \)</strong>: Portfolio weights</li>
            <li><strong>\( w_b \)</strong>: Benchmark weights</li>
            <li><strong>\( \\Sigma \)</strong>: Covariance matrix of sector returns</li>
            <li><strong>\( (w - w_b) \)</strong>: Active weights (portfolio weights minus benchmark weights)</li>
        </ul>

        <h2>Marginal Risk Contribution</h2>
        <p>
            The <strong>Marginal Risk Contribution</strong> represents the sensitivity of the portfolio's active risk to a small change in active weight in sector \( i \). It is calculated as:
        </p>
        <div class="equation">
            \\[
            \\text{{Marginal Risk Contribution}}_i = \\frac{{\\partial \\text{{Ex Ante Active Risk}}}}{{\\partial w_i}} = \\frac{{\\Sigma w}}{{\\text{{Ex Ante Tracking Error}}}}
            \\]
        </div>

        <h2>Ex Ante Active Risk Contribution</h2>
        <p>
            The <strong>Ex Ante Active Risk Contribution</strong> quantifies the risk contributed by each sector to the total Ex Ante active risk. It is calculated as:
        </p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Active Risk Contribution}}_i = \\frac{{\\partial \\text{{Ex Ante Active Risk}}}}{{\\partial w_i}} \\cdot w_i
            \\]
        </div>

        <h2>Objective Function: Minimize Standard Deviation of Ex Ante Active Risk Contributions</h2>
        <p>
            The optimization minimizes the standard deviation of the Ex Ante active risk contributions:
        </p>
        <div class="equation">
            \\[
            \\text{{Objective}} = \\text{{MIN std}}(\\text{{Ex Ante Active Risk Contributions}})
            \\]
        </div>

        <h2>Constraints</h2>
        <ul>
            <li>
                <strong>Active Share Constraint:</strong><br>
                The portfolio must deviate from the benchmark by a specified active share target \( k \) - in our case we set this at 0.5:
                <div class="equation">
                    \\[
                    \\text{{Active Share}} = \\frac{{1}}{{2}} \\sum_{{i=1}}^N |w_i - w_{{b,i}}| = k
                    \\]
                </div>
            </li>
            <li>
                <strong>No Short Constraint:</strong><br>
                Portfolio weights cannot be negative:
                <div class="equation">
                    \\[
                    w_i \\geq 0, \\quad \\forall i
                    \\]
                </div>
            </li>
            <li>
                <strong>Weight Sum Constraint:</strong><br>
                The total sum of weights equals the active share target \( k \):
                <div class="equation">
                    \\[
                    \\sum_{{i=1}}^N w_i = k
                    \\]
                </div>
            </li>
        </ul>
    </div>

    <div class="content">
        <h2>How SLSQP Optimization Works</h2>
        <p>
            The Sequential Least Squares Quadratic Programming (SLSQP) algorithm solves the constrained optimization problem iteratively. 
            The key steps include:
        </p>
        <ul>
            <li>
                <strong>Gradient Calculation:</strong> The gradient of the objective function with respect to the sector weights \( w \) 
                is computed at each iteration.
            </li>
            <li>
                <strong>Constraint Satisfaction:</strong> The algorithm adjusts the sector weights to satisfy all constraints, 
                including the active share and no short constraints.
            </li>
            <li>
                <strong>Step Direction:</strong> Based on the gradient, SLSQP determines the optimal step direction that minimizes 
                the objective function while respecting the constraints.
            </li>
            <li>
                <strong>Convergence:</strong> The algorithm iterates until the change in the objective function or weights is sufficiently small (think 1e-6).
            </li>
        </ul>
    </div>
    </div>

    <div class="content">
        <h2>Example of SLSQP Optimization</h2>
        <p>Below examples are the actual optimizations as of <strong>November 29th 2024</strong>. Iteration 1 represtents the starting position of the optimization and the last iteration is the final solution to the problem. You can use  the sliders to see the progression on how the optimization finds the solution</p>
        <p>The following shows an optimization with <strong>no-short constraint</strong>:</p>
        {plotly_html}
        <p>The following shows the same optimization <strong>allowing short positions</strong>:</p>
        {plotly_html_short}
        <p>Another One</p>
        {html_content}
    </div>

    <div class="content">
        <h2>Numerical Example of Gradient Calculation and Step Size</h2>
        <p>
            To illustrate how the gradient is calculated and the step size is determined in the SLSQP algorithm, we use the following:
        </p>
        <ul>
            <li><strong>Covariance Matrix (\( \\Sigma \))</strong> (as calculated over last 3 years):</li>
        </ul>
        <div class="equation">
        \\[
        \\Sigma = 
        \\begin{{bmatrix}}
        0.089618 & 0.025347 & 0.011072 & 0.015178 & 0.008784 & 0.028007 \\\\
        0.025347 & 0.030405 & 0.029316 & 0.017300 & 0.015910 & 0.029440 \\\\
        0.011072 & 0.029316 & 0.051999 & 0.015003 & 0.019991 & 0.029051 \\\\
        0.015178 & 0.017300 & 0.015003 & 0.033967 & 0.016893 & 0.018864 \\\\
        0.008784 & 0.015910 & 0.019991 & 0.016893 & 0.022112 & 0.018360 \\\\
        0.028007 & 0.029440 & 0.029051 & 0.018864 & 0.018360 & 0.036468
        \\end{{bmatrix}}
        \\]
        </div>
        <p>The <strong>initial guess weights (\( w^0 \))</strong> are equally allocated across sectors:</p>
        <div class="equation">
            \\[
            w^0 = \\begin{{bmatrix}} 0.0833 & 0.0833 & 0.0833 & 0.0833 & 0.0833 & 0.0833 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 1: Multiply \( \\Sigma \) by the Initial Weights \( w^0 \)</h3>
        <p>We calculate \( \\Sigma w^0 \) as follows:</p>
        <div class="equation">
            \\[
            (\\Sigma w^0)_i = \\sum_{{j=1}}^6 \\Sigma_{{i,j}} \\cdot w_j
            \\]
        </div>
        <p>Substituting values for the first row:</p>
        <div class="equation">
            \\[
            (\\Sigma w^0)_1 = (0.089618 \\cdot 0.0833) + (0.025347 \\cdot 0.0833) + \\dots = 0.015
            \\]
        </div>
        <p>This calculation is repeated for all rows, resulting in the vector:</p>
        <div class="equation">
            \\[
            \\Sigma w^0 = \\begin{{bmatrix}} 0.0148 & 0.0123 & 0.0130 & 0.0098 & 0.0085 & 0.0133 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 2: Calculate the Ex Ante Active Risk</h3>
        <p>The total Ex Ante Active Risk (used interchangeably with Ex Ante Tracking Error) is calculated as:</p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Active Risk (or Tracking Error)}} = \\sqrt{{w^T \\Sigma w}}
            \\]
        </div>
        <p>Substituting the values:</p>
        <div class="equation">
            \\[
            w^T \\Sigma w = (0.0833 \\cdot 0.015) + (0.0833 \\cdot 0.012) + \\dots = 0.00598
            \\]
            \\[
            \\text{{Ex Ante Active Risk}} = \\sqrt{{0.00598}} = 0.0773
            \\]
        </div>

        <h3>Step 3: Divide \( \\Sigma w \) by the Ex Ante Active Risk</h3>
        <p>The gradient \( \\nabla f(w) \) is calculated as:</p>
        <div class="equation">
            \\[
            \\nabla f(w) = \\frac{{\\Sigma w}}{{\\text{{Ex Ante Active Risk}}}}
            \\]
        </div>
        <p>Substituting the values:</p>
        <div class="equation">
            \\[
            \\nabla f(w) = \\begin{{bmatrix}}
            \\frac{{0.015}}{{0.077}} & \\frac{{0.012}}{{0.077}} & \\frac{{0.013}}{{0.077}} & \\frac{{0.0098}}{{0.077}} & \\frac{{0.0085}}{{0.077}} & \\frac{{0.0085}}{{0.077}}
            \\end{{bmatrix}}
            \\]
            \\[
            \\nabla f(w) = \\begin{{bmatrix}} 0.191 & 0.159 & 0.168 & 0.126 & 0.110 & 0.173 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 4: Step Size and Direction</h3>
        <p>
            The step size \( \\alpha \) is determined using a line search method, which minimizes the objective function along the calculated gradient direction. \( \\alpha \) represents the 'magnitude' of the step and is calculated iteratively such that constraints are not violated. The algorithm assumes a large \( \\alpha \) initially and incrementally decreases its value by very small steps until no constraints are violated.
            If the maximum size \( \\alpha \)  can be is very small the optimization process terminates.
        </p>
        <p>The updated weights are computed as:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = w^t - \\alpha \\nabla f(w)
            \\]
        </div>
        <p>For example, in the first iteration of the 'with shorts' optimization \( \\alpha = 0.4345 \), the updated weights would be:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = \\begin{{bmatrix}} 0.0833 & 0.0833 & \\dots \\end{{bmatrix}} - 0.4345 \\cdot \\begin{{bmatrix}} 0.191 & 0.159 & \\dots \\end{{bmatrix}}
            \\]
        </div>
        <p>The weights are subsequently bound to zero and scaled to sum to 0.5, which results in weights for the first iteration being:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = \\begin{{bmatrix}} 0 & 0.0502 & 0.0179 & 0.2008 & 0.2311 & 0 \\end{{bmatrix}}
            \\]
        </div>
    </div>
</html>
"""

# Write the file
with open("optimization.html", "w") as file:
    file.write(html_content)

print("Interactive HTML file 'optimization_with_barclays_style.html' created successfully!")


<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

invalid escape sequence '\('

<>:266: SyntaxWarning:

inva

NameError: name 'plotly_html' is not defined

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Objective Function with Adjustable Steepness
def objective_function_with_minima(w1, w2):
    w3 = 1 - w1 - w2
    if w1 < 0 or w2 < 0 or w3 < 0:
        return np.nan  # Exclude invalid points
    return 0.8 * ((w1 - 0.5)**2 + (w2 - 0.25)**2 + (w3 - 0.25)**2) + 0.03 * (np.sin(5 * w1) + np.sin(5 * w2))

# Grid for Surface Plot
w1_range = np.linspace(0, 1, 100)
w2_range = np.linspace(0, 1, 100)
w1_grid, w2_grid = np.meshgrid(w1_range, w2_range)

# Calculate Objective Values
objective_values = np.zeros_like(w1_grid)
for i in range(w1_grid.shape[0]):
    for j in range(w1_grid.shape[1]):
        objective_values[i, j] = objective_function_with_minima(w1_grid[i, j], w2_grid[i, j])
objective_values = np.ma.masked_invalid(objective_values)

# SLSQP Iteration Path (Simulated Steps)
slsqp_path = [
    (0.07, 0.73), (0.21, 0.60), (0.25, 0.55), (0.33, 0.49), (0.38, 0.40), (0.52, 0.26), (0.58, 0.18)
]
w1_path, w2_path = zip(*slsqp_path)
objective_path = [objective_function_with_minima(w1, w2) for w1, w2 in slsqp_path]

# Frames for Animation
frames = []
for i in range(1, len(slsqp_path) + 1):
    frames.append(
        go.Frame(
            data=[
                go.Surface(
                    z=objective_values,
                    x=w1_range,
                    y=w2_range,
                    colorscale="Plasma",
                    opacity=0.9,
                    showscale=False
                ),
                go.Scatter3d(
                    x=w1_path[:i],
                    y=w2_path[:i],
                    z=objective_path[:i],
                    mode="markers+lines",
                    marker=dict(size=6, color="red"),
                    line=dict(color="red", width=3),
                ),
                go.Scatter3d(
                    x=[w1_path[-1]],
                    y=[w2_path[-1]],
                    z=[objective_path[-1]],
                    mode="markers",
                    marker=dict(size=10, color="green", symbol="diamond"),
                    name="True Minimum"
                )
            ],
            name=f"Step {i}"
        )
    )

# Build Figure
fig = go.Figure(
    data=[
        go.Surface(
            z=objective_values,
            x=w1_range,
            y=w2_range,
            colorscale="Plasma",
            opacity=0.9,
            showscale=False,
        ),
        go.Scatter3d(
            x=[w1_path[0]],
            y=[w2_path[0]],
            z=[objective_path[0]],
            mode="markers",
            marker=dict(size=6, color="red"),
            name="SQLSP Iteration"
        ),
    ],
    layout=go.Layout(
        title="SLSQP Optimization Graphic",
        width=1200,
        height=800,
        scene=dict(
            xaxis=dict(title="Sector 1 (Weight %)", range=[0, 1], tickformat=".0%"),
            yaxis=dict(title="Sector 2 (Weight %)", range=[0, 1], tickformat=".0%"),
            zaxis=dict(title="Variance in Risk Contribution", range=[0, 0.4]),
        ),
        updatemenus=[
            {
                "buttons": [
                    {"args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}],
                     "label": "Play Animation", "method": "animate"},
                    {"args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}],
                     "label": "Pause Animation", "method": "animate"},
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": True,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "left",
                "y": -0.2,
                "yanchor": "top",
            }
        ],
    ),
    frames=frames
)

# Export HTML
html_content = fig.to_html(full_html=False, include_plotlyjs="cdn")
with open("steeper_slsqp_optimization.html", "w") as file:
    file.write(html_content)

print("3D Animated Optimization with Steeper Surface saved as 'steeper_slsqp_optimization.html'")


3D Animated Optimization with Steeper Surface saved as 'steeper_slsqp_optimization.html'


In [44]:


html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Optimization Visualization</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-AMS_CHTML"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            color: #333;
            margin: 0;
            padding: 20px;
        }}
        h1, h2, h3 {{
            color: #0078D7;
        }}
        .content {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 4px 6px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }}
        p, ul {{
            line-height: 1.6;
        }}
        ul {{
            padding-left: 20px;
        }}
        .equation {{
            text-align: center;
            margin: 10px 0;
        }}
    </style>
</head>
<body>
    <div class="content">
        <h1>Mathematical Explanation of the Risk Distribution Optimization</h1>

        <h2>Ex Ante Active Risk</h2>
        <p>
            The <strong>Ex Ante Active Risk</strong> (used interchangeably with Ex Ante Tracking Error) measures the portfolio's <strong>expected</strong>  deviation from the benchmark. It is defined as:
        </p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Tracking Error}} = \\sqrt{{(w - w_b)^T \\Sigma (w - w_b)}}
            \\]
        </div>
        <p>Where:</p>
        <ul>
            <li><strong>\( w \)</strong>: Portfolio weights</li>
            <li><strong>\( w_b \)</strong>: Benchmark weights</li>
            <li><strong>\( \\Sigma \)</strong>: Covariance matrix of sector returns</li>
            <li><strong>\( (w - w_b) \)</strong>: Active weights (portfolio weights minus benchmark weights)</li>
        </ul>

        <h2>Marginal Risk Contribution</h2>
        <p>
            The <strong>Marginal Risk Contribution</strong> represents the sensitivity of the portfolio's active risk to a small change in active weight in sector \( i \). It is calculated as:
        </p>
        <div class="equation">
            \\[
            \\text{{Marginal Risk Contribution}}_i = \\frac{{\\partial \\text{{Ex Ante Active Risk}}}}{{\\partial w_i}} = \\frac{{\\Sigma w}}{{\\text{{Ex Ante Tracking Error}}}}
            \\]
        </div>

        <h2>Ex Ante Active Risk Contribution</h2>
        <p>
            The <strong>Ex Ante Active Risk Contribution</strong> quantifies the risk contributed by each sector to the total Ex Ante active risk. It is calculated as:
        </p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Active Risk Contribution}}_i = \\frac{{\\partial \\text{{Ex Ante Active Risk}}}}{{\\partial w_i}} \\cdot w_i
            \\]
        </div>

        <h2>Objective Function: Minimize Standard Deviation of Ex Ante Active Risk Contributions</h2>
        <p>
            The optimization minimizes the standard deviation of the Ex Ante active risk contributions:
        </p>
        <div class="equation">
            \\[
            \\text{{Objective}} = \\text{{MIN std}}(\\text{{Ex Ante Active Risk Contributions}})
            \\]
        </div>

        <h2>Constraints</h2>
        <ul>
            <li>
                <strong>Active Share Constraint:</strong><br>
                The portfolio must deviate from the benchmark by a specified active share target \( k \) - in our case we set this at 0.5:
                <div class="equation">
                    \\[
                    \\text{{Active Share}} = \\frac{{1}}{{2}} \\sum_{{i=1}}^N |w_i - w_{{b,i}}| = k
                    \\]
                </div>
            </li>
            <li>
                <strong>No Short Constraint:</strong><br>
                Portfolio weights cannot be negative:
                <div class="equation">
                    \\[
                    w_i \\geq 0, \\quad \\forall i
                    \\]
                </div>
            </li>
            <li>
                <strong>Weight Sum Constraint:</strong><br>
                The total sum of weights equals the active share target \( k \):
                <div class="equation">
                    \\[
                    \\sum_{{i=1}}^N w_i = k
                    \\]
                </div>
            </li>
        </ul>
    </div>

    <div class="content">
        <h2>How SLSQP Optimization Works</h2>
        <p>
            The Sequential Least Squares Quadratic Programming (SLSQP) algorithm solves the constrained optimization problem iteratively. 
            The key steps include:
        </p>
        <ul>
            <li>
                <strong>Gradient Calculation:</strong> The gradient of the objective function with respect to the sector weights \( w \) 
                is computed at each iteration.
            </li>
            <li>
                <strong>Constraint Satisfaction:</strong> The algorithm adjusts the sector weights to satisfy all constraints, 
                including the active share and no short constraints.
            </li>
            <li>
                <strong>Step Direction:</strong> Based on the gradient, SLSQP determines the optimal step direction that minimizes 
                the objective function while respecting the constraints.
            </li>
            <li>
                <strong>Convergence:</strong> The algorithm iterates until the change in the objective function or weights is sufficiently small (think 1e-6).
            </li>
        </ul>
    </div>
    </div>

    <div class="content">
        <h2>Example of SLSQP Optimization</h2>
        <p>Below examples are the actual optimizations as of <strong>November 29th 2024</strong>. Iteration 1 represtents the starting position of the optimization and the last iteration is the final solution to the problem. You can use  the sliders to see the progression on how the optimization finds the solution</p>
        <p>The following shows an optimization with <strong>no-short constraint</strong>:</p>
        <p>The following shows the same optimization <strong>allowing short positions</strong>:</p>
        <p>Another One</p>
        {html_content}
    </div>

    <div class="content">
        <h2>Numerical Example of Gradient Calculation and Step Size</h2>
        <p>
            To illustrate how the gradient is calculated and the step size is determined in the SLSQP algorithm, we use the following:
        </p>
        <ul>
            <li><strong>Covariance Matrix (\( \\Sigma \))</strong> (as calculated over last 3 years):</li>
        </ul>
        <div class="equation">
        \\[
        \\Sigma = 
        \\begin{{bmatrix}}
        0.089618 & 0.025347 & 0.011072 & 0.015178 & 0.008784 & 0.028007 \\\\
        0.025347 & 0.030405 & 0.029316 & 0.017300 & 0.015910 & 0.029440 \\\\
        0.011072 & 0.029316 & 0.051999 & 0.015003 & 0.019991 & 0.029051 \\\\
        0.015178 & 0.017300 & 0.015003 & 0.033967 & 0.016893 & 0.018864 \\\\
        0.008784 & 0.015910 & 0.019991 & 0.016893 & 0.022112 & 0.018360 \\\\
        0.028007 & 0.029440 & 0.029051 & 0.018864 & 0.018360 & 0.036468
        \\end{{bmatrix}}
        \\]
        </div>
        <p>The <strong>initial guess weights (\( w^0 \))</strong> are equally allocated across sectors:</p>
        <div class="equation">
            \\[
            w^0 = \\begin{{bmatrix}} 0.0833 & 0.0833 & 0.0833 & 0.0833 & 0.0833 & 0.0833 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 1: Multiply \( \\Sigma \) by the Initial Weights \( w^0 \)</h3>
        <p>We calculate \( \\Sigma w^0 \) as follows:</p>
        <div class="equation">
            \\[
            (\\Sigma w^0)_i = \\sum_{{j=1}}^6 \\Sigma_{{i,j}} \\cdot w_j
            \\]
        </div>
        <p>Substituting values for the first row:</p>
        <div class="equation">
            \\[
            (\\Sigma w^0)_1 = (0.089618 \\cdot 0.0833) + (0.025347 \\cdot 0.0833) + \\dots = 0.015
            \\]
        </div>
        <p>This calculation is repeated for all rows, resulting in the vector:</p>
        <div class="equation">
            \\[
            \\Sigma w^0 = \\begin{{bmatrix}} 0.0148 & 0.0123 & 0.0130 & 0.0098 & 0.0085 & 0.0133 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 2: Calculate the Ex Ante Active Risk</h3>
        <p>The total Ex Ante Active Risk (used interchangeably with Ex Ante Tracking Error) is calculated as:</p>
        <div class="equation">
            \\[
            \\text{{Ex Ante Active Risk (or Tracking Error)}} = \\sqrt{{w^T \\Sigma w}}
            \\]
        </div>
        <p>Substituting the values:</p>
        <div class="equation">
            \\[
            w^T \\Sigma w = (0.0833 \\cdot 0.015) + (0.0833 \\cdot 0.012) + \\dots = 0.00598
            \\]
            \\[
            \\text{{Ex Ante Active Risk}} = \\sqrt{{0.00598}} = 0.0773
            \\]
        </div>

        <h3>Step 3: Divide \( \\Sigma w \) by the Ex Ante Active Risk</h3>
        <p>The gradient \( \\nabla f(w) \) is calculated as:</p>
        <div class="equation">
            \\[
            \\nabla f(w) = \\frac{{\\Sigma w}}{{\\text{{Ex Ante Active Risk}}}}
            \\]
        </div>
        <p>Substituting the values:</p>
        <div class="equation">
            \\[
            \\nabla f(w) = \\begin{{bmatrix}}
            \\frac{{0.015}}{{0.077}} & \\frac{{0.012}}{{0.077}} & \\frac{{0.013}}{{0.077}} & \\frac{{0.0098}}{{0.077}} & \\frac{{0.0085}}{{0.077}} & \\frac{{0.0085}}{{0.077}}
            \\end{{bmatrix}}
            \\]
            \\[
            \\nabla f(w) = \\begin{{bmatrix}} 0.191 & 0.159 & 0.168 & 0.126 & 0.110 & 0.173 \\end{{bmatrix}}
            \\]
        </div>

        <h3>Step 4: Step Size and Direction</h3>
        <p>
            The step size \( \\alpha \) is determined using a line search method, which minimizes the objective function along the calculated gradient direction. \( \\alpha \) represents the 'magnitude' of the step and is calculated iteratively such that constraints are not violated. The algorithm assumes a large \( \\alpha \) initially and incrementally decreases its value by very small steps until no constraints are violated.
            If the maximum size \( \\alpha \)  can be is very small the optimization process terminates.
        </p>
        <p>The updated weights are computed as:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = w^t - \\alpha \\nabla f(w)
            \\]
        </div>
        <p>For example, in the first iteration of the 'with shorts' optimization \( \\alpha = 0.4345 \), the updated weights would be:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = \\begin{{bmatrix}} 0.0833 & 0.0833 & \\dots \\end{{bmatrix}} - 0.4345 \\cdot \\begin{{bmatrix}} 0.191 & 0.159 & \\dots \\end{{bmatrix}}
            \\]
        </div>
        <p>The weights are subsequently bound to zero and scaled to sum to 0.5, which results in weights for the first iteration being:</p>
        <div class="equation">
            \\[
            w^{{t+1}} = \\begin{{bmatrix}} 0 & 0.0502 & 0.0179 & 0.2008 & 0.2311 & 0 \\end{{bmatrix}}
            \\]
        </div>
    </div>
</html>
"""

# Write the file
with open("optimization.html", "w") as file:
    file.write(html_content)

print("Interactive HTML file 'optimization_with_barclays_style.html' created successfully!")


Interactive HTML file 'optimization_with_barclays_style.html' created successfully!


<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

invalid escape sequence '\('

<>:264: SyntaxWarning:

inva